In [1]:
import sys 
sys.path.append(rf"/nfs/home/genovese/thesis-wildfire-genovese/src")
from importlib import reload
import utils
reload(utils)
from utils import *
data_folder = "/nfs/home/genovese/thesis-wildfire-genovese/data/"
output_folder = "/nfs/home/genovese/thesis-wildfire-genovese/data/gathering/per_comune/"

In [2]:
sesso9701 = pd.read_csv(data_folder+'clean_data/popolazione_piemonte/population_1997_2001_per_sex.csv.csv')
eta0219 = pd.read_csv(data_folder+'clean_data/popolazione_piemonte/population_2002_2019_per_age.csv.csv')
eta0211 = pd.read_csv(data_folder+'clean_data/popolazione_piemonte/population_2002_2011_per_age.csv.csv')
sesso1924 = pd.read_csv(data_folder+'clean_data/popolazione_piemonte/population_2019_2024_per_sex.csv.csv')

In [3]:
confini = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/clean_data/confini_piemonte/confini_piemonte.shp')

---

In [5]:
sesso9701.drop(columns=sesso9701.columns[[0,1]], inplace=True)

In [6]:
pop_97_01 = sesso9701.merge(confini.rename(columns={'Codice Com': 'Codice comune'}), on='Codice comune', how='inner')

In [7]:
pop_97_01.drop(columns=pop_97_01.columns[[0, 1, 10, 11]], inplace=True)

In [9]:
dic_97_01 = dict.fromkeys(list(pop_97_01.Anno.unique()))
for a in dic_97_01.keys():
    dic_97_01[a] = pop_97_01[pop_97_01['Anno'] == a].drop(columns=['Anno'])


In [11]:
for a, df in dic_97_01.items():
    lst = []
    for geom in df.geometry.unique():
        lst.append(df[df.geometry == geom].shape[0])
    print(pd.Series(lst).unique())

[3]
[3]
[3]
[3]
[3]


For each geometry I have "Maschi, Femmine, Totale"

In [38]:
for a, df in dic_97_01.items():
    foo = df.pivot_table(columns='Sesso', index='geometry')
    foo.columns = ['_'.join(col).strip() for col in foo.columns.values]
    foo.reset_index()
    dic_97_01[a] = foo

---

In [45]:
eta0219

,Unnamed: 0.1,Unnamed: 0,2002,2003,2004,2005,2006,2007,2008,2009,...,2012,2013,2014,2015,2016,2017,2018,2019,Eta,Comune
0,0,0,18,17,24,17,17,13,21,28,...,21,20,30,17,20,17,13,23,0,A001001
1,1,1,19,16,20,28,17,20,11,19,...,20,20,25,26,21,22,19,12,1,A001001
2,2,2,21,21,19,21,30,17,17,15,...,21,22,22,23,28,21,25,19,2,A001001
3,3,3,19,21,22,16,21,30,17,19,...,25,23,24,23,23,31,22,25,3,A001001
4,4,4,16,21,20,22,15,22,26,17,...,24,28,26,26,19,26,31,23,4,A001001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120253,120253,120253,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,97,A103079
120254,120254,120254,0,0,1,0,0,0,0,0,...,1,0,0,0,1,0,0,0,98,A103079
120255,120255,120255,0,0,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,99,A103079
120256,120256,120256,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,100 e oltre,A103079


In [50]:
pop_02_19 = eta0219.drop(columns = eta0219.columns[:2]).merge(confini[['Codice Com', 'geometry']].rename(columns={'Codice Com': 'Comune'}), on='Comune', how='inner').drop(columns='Comune')

In [51]:
pop_02_19

,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Eta,geometry
0,18,17,24,17,17,13,21,28,17,19,21,20,30,17,20,17,13,23,0,"POLYGON ((7.78266 45.38717, 7.79106 45.3863, 7..."
1,19,16,20,28,17,20,11,19,25,16,20,20,25,26,21,22,19,12,1,"POLYGON ((7.78266 45.38717, 7.79106 45.3863, 7..."
2,21,21,19,21,30,17,17,15,21,26,21,22,22,23,28,21,25,19,2,"POLYGON ((7.78266 45.38717, 7.79106 45.3863, 7..."
3,19,21,22,16,21,30,17,19,15,23,25,23,24,23,23,31,22,25,3,"POLYGON ((7.78266 45.38717, 7.79106 45.3863, 7..."
4,16,21,20,22,15,22,26,17,20,15,24,28,26,26,19,26,31,23,4,"POLYGON ((7.78266 45.38717, 7.79106 45.3863, 7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119131,0,0,1,0,1,0,0,2,0,0,0,0,2,0,0,0,1,0,97,"POLYGON ((8.27423 46.04232, 8.3037 46.03906, 8..."
119132,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,98,"POLYGON ((8.27423 46.04232, 8.3037 46.03906, 8..."
119133,0,1,0,0,0,0,2,0,0,1,0,1,0,0,1,0,0,0,99,"POLYGON ((8.27423 46.04232, 8.3037 46.03906, 8..."
119134,0,0,1,1,0,0,0,1,0,0,1,0,1,0,0,1,1,1,100 e oltre,"POLYGON ((8.27423 46.04232, 8.3037 46.03906, 8..."
